# 空調最適化システム - 段階的プロセス確認・改善ノートブック

このノートブックでは、前処理後データから電力・環境予測モデル構築、制御値探索、結果出力までのプロセスを段階的に確認・改善します。

## プロセス概要
1. **データ読み込み・確認** - 前処理済みデータの内容確認
2. **特徴量エンジニアリング** - モデル学習用特徴量の準備
3. **予測モデル構築** - 電力・環境予測モデルの学習・評価
4. **制御値探索** - 最適制御値の探索・評価
5. **結果出力・可視化** - 最適化結果の出力・分析

各ステップで中間結果を確認し、必要に応じて改善を行います。


In [1]:
# 必要なライブラリのインポート
import os
import sys
import time
import warnings
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import joblib
import jpholiday  # 祝日検出用

# プロジェクトのルートディレクトリをパスに追加
sys.path.append('.')

# モジュールの再読み込みを有効化
import importlib

# 設定とユーティリティのインポート
from config.utils import load_config, get_data_path

# # モジュールを再読み込み（最新のコードを反映）
# try:
#     importlib.reload(sys.modules['processing.preprocessor'])
# except KeyError:
#     pass  # まだ読み込まれていない場合はスキップ

from processing.preprocessor import DataPreprocessor

from processing.aggregator import AreaAggregator
from training.model_builder import ModelBuilder, EnvPowerModels
from optimization.period_optimizer import PeriodOptimizer
from planning.planner import Planner

# 警告を非表示
warnings.filterwarnings('ignore')

print("✅ ライブラリのインポート完了")


✅ ライブラリのインポート完了


## 1. 前処理プロセス

生データから前処理済みデータまでのプロセスを確認します。

### 1.1 生データの読み込み・確認


In [2]:
# 設定の読み込み
config = load_config()
store_name = "Clea"

# マスターデータの読み込み
master_path = f"data/01_MasterData/MASTER_{store_name}.json"
import json
with open(master_path, 'r', encoding='utf-8') as f:
    master = json.load(f)

print(f"🏢 ストア: {store_name}")
print(f"📊 マスターデータ: {list(master.keys())}")
print(f"🏢 ゾーン数: {len(master.get('zones', {}))}")
print(f"🌍 座標: {master.get('store_info', {}).get('coordinates', 'N/A')}")

# 前処理器の初期化
preprocessor = DataPreprocessor(store_name)
print(f"📁 データディレクトリ: {preprocessor.data_dir}")
print(f"📁 出力ディレクトリ: {preprocessor.output_dir}")


🏢 ストア: Clea
📊 マスターデータ: ['store_info', 'zones']
🏢 ゾーン数: 6
🌍 座標: 35.681236%2C139.767125
📁 データディレクトリ: /Users/tomokiinoue/Desktop/workspace/AIrux8_opti_logic/data/00_InputData/Clea
📁 出力ディレクトリ: /Users/tomokiinoue/Desktop/workspace/AIrux8_opti_logic/data/02_PreprocessedData/Clea


In [3]:
# 生データの読み込み
print("🔍 生データの読み込み...")
ac_raw, pm_raw = preprocessor.load_raw()

if ac_raw is not None:
    print(f"✅ AC制御データ読み込み完了: {ac_raw.shape}")
    print(f"📋 AC制御データの列: {list(ac_raw.columns)}")
    print(f"📅 AC制御データ期間: {ac_raw['datetime'].min() if 'datetime' in ac_raw.columns else 'N/A'} ～ {ac_raw['datetime'].max() if 'datetime' in ac_raw.columns else 'N/A'}")
    
    # カテゴリカル変数の確認
    print(f"\n🔍 AC制御データのカテゴリカル変数確認:")
    for col in ["A/C ON/OFF", "A/C Mode", "A/C Fan Speed"]:
        if col in ac_raw.columns:
            unique_vals = ac_raw[col].unique()
            print(f"  {col}: {unique_vals}")
else:
    print("❌ AC制御データが見つかりません")

if pm_raw is not None:
    print(f"✅ 電力メーターデータ読み込み完了: {pm_raw.shape}")
    print(f"📋 電力メーターデータの列: {list(pm_raw.columns)}")
    print(f"📅 電力メーターデータ期間: {pm_raw['datetime'].min() if 'datetime' in pm_raw.columns else 'N/A'} ～ {pm_raw['datetime'].max() if 'datetime' in pm_raw.columns else 'N/A'}")
    
    # 電力データの確認
    if "Phase A" in pm_raw.columns:
        power_stats = pm_raw["Phase A"].describe()
        print(f"\n⚡ 電力データ統計:")
        print(f"  平均: {power_stats['mean']:.1f}W")
        print(f"  最大: {power_stats['max']:.1f}W")
        print(f"  最小: {power_stats['min']:.1f}W")
        print(f"  欠損値: {pm_raw['Phase A'].isnull().sum()}件")
else:
    print("❌ 電力メーターデータが見つかりません")

🔍 生データの読み込み...
[DataPreprocessor] Loading raw data from: /Users/tomokiinoue/Desktop/workspace/AIrux8_opti_logic/data/00_InputData/Clea
[DataPreprocessor] Directory exists: True
[DataPreprocessor] Found 18 AC control files
[DataPreprocessor] Found 24 power meter files
[DataPreprocessor] AC files: ['/Users/tomokiinoue/Desktop/workspace/AIrux8_opti_logic/data/00_InputData/Clea/ac-control/ac-control-50,66,67,65,59,63,57,68,69,75,49,56,52,64,54,53,55,58,62,60,51,373-logs-2025-07-01-2025-07-31.csv']...
[DataPreprocessor] PM files: ['/Users/tomokiinoue/Desktop/workspace/AIrux8_opti_logic/data/00_InputData/Clea/ac-power-meter/ac-power-meter-44,49,44,49,49,49,43,44,44,49,41,43,44,49,43,44,43,49,49,49,44-7,9,2,1,7,4,1,4,4,3,1,3,5,2,4,1,2,8,6,4,3-logs-2025-05-01-2025-05-31.csv']...
[DataPreprocessor] AC data shape: (2818340, 13)
[DataPreprocessor] PM data shape: (10705341, 6)
✅ AC制御データ読み込み完了: (2818340, 13)
📋 AC制御データの列: ['A/C Name', 'Datetime', 'Outdoor Temp.', 'Indoor Temp.', 'A/C Set Temperature

In [4]:

ac_raw[[str(i)=="nan" for i in ac_raw["A/C Mode"]]]
ac_columns = [i for i in ac_raw.columns if "A/C" in i]
ac_raw[ac_columns] = ac_raw[ac_columns].replace(np.nan,"-")

for ac_column in ac_columns:
    print(f"")
    print(f"【{ac_column}】")
    print(ac_raw[ac_column].value_counts())



【A/C Name】
D-1南1     121852
D-3南2     121852
A-26      121852
D-2北1     121852
E-11南3    121852
E-13北1    121852
E-9南1     121852
D-6北1     121852
D-4北2     121852
D-5南1     121852
E-17      121852
E-15北3    121852
D-7南2     121852
A-25      121852
E-12南4    121852
F-20      121852
F-19      121852
D-8北2     121852
E-14北2    121852
E-10南2    121852
E-16北4    121852
F-18      121852
G-24       34399
G-21       34399
G-22       34399
G-23       34399
Name: A/C Name, dtype: int64

【A/C Set Temperature】
25.0    1297750
26.0     584037
24.0     327427
27.0     198143
22.0     154614
20.0      97050
-         58440
23.0      49251
21.0      36465
16.0       6883
28.0       2629
30.0       1584
29.0       1490
19.0       1425
18.0        542
31.0        287
33.0        203
32.0        117
34.0          3
Name: A/C Set Temperature, dtype: int64

【A/C ON/OFF】
OFF    1953566
ON      864774
Name: A/C ON/OFF, dtype: int64

【A/C Mode】
COOL    1583968
HEAT     923191
FAN      209766
-        101415

### 1.2 前処理の実行

生データを前処理して、カテゴリカル変数の変換と欠損値処理を行います。


In [5]:
# 元データのカテゴリカル変数を詳細調査
print("🔍 元データのカテゴリカル変数詳細調査...")
if ac_raw is not None:
    for col in ["A/C ON/OFF", "A/C Mode", "A/C Fan Speed"]:
        if col in ac_raw.columns:
            print(f"\n📊 {col} の詳細分析:")
            unique_vals = ac_raw[col].value_counts()
            print(f"  値の分布:")
            for val, count in unique_vals.head(10).items():
                print(f"    '{val}': {count}件")
            
            # データ型の確認
            print(f"  データ型: {ac_raw[col].dtype}")
            
            # 欠損値の確認
            missing_count = ac_raw[col].isnull().sum()
            print(f"  欠損値: {missing_count}件")
            
            # 空白文字の確認
            if ac_raw[col].dtype == 'object':
                whitespace_count = ac_raw[col].str.strip().eq('').sum()
                print(f"  空白文字: {whitespace_count}件")
                
                # 特殊文字の確認
                special_chars = ac_raw[col].str.contains(r'[^\w\s-]', na=False).sum()
                print(f"  特殊文字を含む値: {special_chars}件")
                
                # 数値以外の値の確認
                non_numeric = ac_raw[col].str.match(r'^\d+$', na=False).sum()
                print(f"  数値形式: {non_numeric}件 / 全{len(ac_raw)}件")

# AC制御データの前処理（エリア別マッピング使用）
print("\n🔧 AC制御データの前処理（エリア別マッピング）...")
ac_processed = preprocessor.preprocess_ac(
    ac_raw, 
    standard_deviation_multiplier=5.0,
    category_mapping=None,  # 共通マッピングは使用しない
    zone_specific_mapping=True  # エリア別マッピングを有効化
)

if ac_processed is not None:
    print(f"✅ AC制御データ前処理完了: {ac_processed.shape}")
    
    # エリア別マッピングログの確認
    import glob
    log_files = glob.glob(f"logs/preprocessing/{store_name}/zone_mapping_log_*.json")
    if log_files:
        latest_log = max(log_files, key=os.path.getctime)
        print(f"\n📋 エリア別マッピングログ: {latest_log}")
        
        # ログファイルの内容を表示
        import json
        with open(latest_log, 'r', encoding='utf-8') as f:
            log_data = json.load(f)
        
        print(f"\n🔍 エリア別マッピング結果:")
        for zone_name, zone_info in log_data["zones"].items():
            print(f"\n📍 {zone_name}:")
            print(f"  レコード数: {zone_info['total_records']}")
            
            for col, mapping_info in zone_info["categorical_mappings"].items():
                print(f"  {col}:")
                print(f"    元の値: {list(mapping_info['original_values'].keys())}")
                print(f"    マッピング: {mapping_info['mapping']}")
                if mapping_info.get('unmapped_count', 0) > 0:
                    print(f"    未マッピング: {mapping_info['unmapped_count']}件")
                    print(f"    デフォルト値: {mapping_info.get('default_value', 'N/A')}")
    
else:
    print("❌ AC制御データの前処理に失敗")


🔍 元データのカテゴリカル変数詳細調査...

📊 A/C ON/OFF の詳細分析:
  値の分布:
    'OFF': 1953566件
    'ON': 864774件
  データ型: object
  欠損値: 0件
  空白文字: 0件
  特殊文字を含む値: 0件
  数値形式: 0件 / 全2818340件

📊 A/C Mode の詳細分析:
  値の分布:
    'COOL': 1583968件
    'HEAT': 923191件
    'FAN': 209766件
    '-': 101415件
  データ型: object
  欠損値: 0件
  空白文字: 0件
  特殊文字を含む値: 0件
  数値形式: 0件 / 全2818340件

📊 A/C Fan Speed の詳細分析:
  値の分布:
    '-': 1953566件
    'Low': 665506件
    'Medium': 163051件
    'High': 34844件
    'Auto': 1370件
    'Top': 3件
  データ型: object
  欠損値: 0件
  空白文字: 0件
  特殊文字を含む値: 0件
  数値形式: 0件 / 全2818340件

🔧 AC制御データの前処理（エリア別マッピング）...
[DataPreprocessor] カテゴリカル変数のマッピングは後段階で実行します
✅ AC制御データ前処理完了: (2759789, 14)


In [6]:
ac_processed

,Datetime,Date,A/C Name,Outdoor Temp.,Indoor Temp.,A/C Set Temperature,A/C ON/OFF,A/C Mode,A/C Fan Speed,Naive Energy Level,Airux Energy Level,Outdoor Room Temp.,Outdoor Set Temp.,Room Set Temp.
0,2025-07-31 14:55:00,2025-07-31,D-1南1,20.0,26.0,24.0,ON,COOL,Low,62,32,-6.0,-4.0,2.0
1,2025-07-31 14:55:00,2025-07-31,E-17,20.0,32.0,27.0,OFF,COOL,-,0,0,-12.0,-7.0,5.0
2,2025-07-31 14:55:00,2025-07-31,F-18,20.0,30.0,26.0,OFF,COOL,-,0,0,-10.0,-6.0,4.0
3,2025-07-31 14:55:00,2025-07-31,E-16北4,20.0,27.0,24.0,OFF,COOL,-,0,0,-7.0,-4.0,3.0
4,2025-07-31 14:55:00,2025-07-31,E-10南2,20.0,27.0,24.0,OFF,COOL,-,0,0,-7.0,-4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818335,2024-10-31 15:00:00,2024-10-31,E-15北3,10.0,18.0,20.0,OFF,FAN,-,0,0,-8.0,-10.0,-2.0
2818336,2024-10-31 15:00:00,2024-10-31,D-4北2,10.0,25.0,26.0,OFF,FAN,-,0,0,-15.0,-16.0,-1.0
2818337,2024-10-31 15:00:00,2024-10-31,D-5南1,10.0,23.0,24.0,ON,COOL,Medium,32,32,-13.0,-14.0,-1.0
2818338,2024-10-31 15:00:00,2024-10-31,E-11南3,10.0,17.0,25.0,OFF,FAN,-,0,0,-7.0,-15.0,-8.0


In [7]:
# # -1の値の処理改善
# print("\n🔧 -1の値の処理改善...")
# if ac_processed is not None:
#     for col in ["A/C ON/OFF", "A/C Mode", "A/C Fan Speed"]:
#         if col in ac_processed.columns:
#             minus_one_count = (ac_processed[col] == -1).sum()
#             if minus_one_count > 0:
#                 print(f"\n⚠️ {col} に-1の値が {minus_one_count}件 あります")
                
#                 # 適切なデフォルト値に置換
#                 if col == "A/C ON/OFF":
#                     default_value = 0  # OFF
#                     default_name = "OFF"
#                 elif col == "A/C Mode":
#                     default_value = 2  # FAN
#                     default_name = "FAN"
#                 elif col == "A/C Fan Speed":
#                     default_value = 1  # Low
#                     default_name = "Low"
#                 else:
#                     default_value = 0
#                     default_name = "Unknown"
                
#                 print(f"  {default_name}({default_value})に置換します...")
#                 ac_processed[col] = ac_processed[col].replace(-1, default_value)
#                 print(f"  ✅ 置換完了: {minus_one_count}件 → {default_name}({default_value})")

# 電力メーターデータの前処理
print("\n🔧 電力メーターデータの前処理...")
pm_processed = preprocessor.preprocess_pm(
    pm_raw, 
    standard_deviation_multiplier=5.0
)

if pm_processed is not None:
    print(f"✅ 電力メーターデータ前処理完了: {pm_processed.shape}")
    
    # 電力データの統計確認
    if "Phase A" in pm_processed.columns:
        power_stats = pm_processed["Phase A"].describe()
        print(f"\n⚡ 前処理後電力データ統計:")
        print(f"  平均: {power_stats['mean']:.1f}W")
        print(f"  最大: {power_stats['max']:.1f}W")
        print(f"  最小: {power_stats['min']:.1f}W")
        print(f"  欠損値: {pm_processed['Phase A'].isnull().sum()}件")
        
        # ゼロ値の確認
        zero_count = (pm_processed["Phase A"] == 0).sum()
        print(f"  ゼロ値: {zero_count}件 ({zero_count/len(pm_processed)*100:.1f}%)")
else:
    print("❌ 電力メーターデータの前処理に失敗")



🔧 電力メーターデータの前処理...
✅ 電力メーターデータ前処理完了: (2159566, 8)

⚡ 前処理後電力データ統計:
  平均: 448.5W
  最大: 4850.0W
  最小: 0.0W
  欠損値: 0件
  ゼロ値: 729件 (0.0%)


In [8]:
pm_processed

,Datetime,Date,Mesh ID,PM Addr ID,Phase A,Phase B,Phase C,Total_kWh
0,2025-05-31 14:59:00,2025-05-31,44,2,10,0,0,10
6,2025-05-31 14:59:00,2025-05-31,49,9,75,0,0,75
14,2025-05-31 14:59:00,2025-05-31,41,1,2,0,0,2
15,2025-05-31 14:59:00,2025-05-31,43,4,65,0,0,65
19,2025-05-31 14:58:00,2025-05-31,44,4,45,0,0,45
...,...,...,...,...,...,...,...,...
10619499,2024-08-31 15:01:00,2024-08-31,41,1,2,0,0,2
10619500,2024-08-31 15:01:00,2024-08-31,43,3,25,0,0,25
10619510,2024-08-31 15:00:00,2024-08-31,49,6,65,0,0,65
10619518,2024-08-31 15:00:00,2024-08-31,41,1,2,0,0,2


In [9]:
# 制御エリア集約
print("\n🔧 制御エリア集約...")
aggregator = AreaAggregator(master)

# 天気データは後で追加するため、ここではNone
area_df = aggregator.build(
    ac_processed, pm_processed, None, freq="1H"
)

if area_df is not None and not area_df.empty:
    print(f"✅ 制御エリア集約完了: {area_df.shape}")
    print(f"🏢 ゾーン: {area_df['zone'].unique().tolist()}")
    
    # 祝日検出の確認
    if "IsHoliday" in area_df.columns:
        holiday_count = area_df["IsHoliday"].sum()
        total_count = len(area_df)
        print(f"\n🎌 祝日検出結果:")
        print(f"  祝日データ: {holiday_count}件 ({holiday_count/total_count*100:.1f}%)")
        
        # 祝日データのサンプル表示
        holiday_dates = area_df[area_df["IsHoliday"] == 1]["Datetime"].dt.date.unique()
        print(f"  祝日一覧: {sorted(holiday_dates)[:10]}...")  # 最初の10件
        
        # 日本の祝日ライブラリとの比較
        print(f"\n🔍 祝日検出の検証:")
        sample_dates = area_df["Datetime"].dt.date.unique()[:5]
        for date in sample_dates:
            is_holiday_lib = jpholiday.is_holiday(date)
            is_holiday_data = area_df[area_df["Datetime"].dt.date == date]["IsHoliday"].iloc[0] if len(area_df[area_df["Datetime"].dt.date == date]) > 0 else 0
            print(f"  {date}: ライブラリ={is_holiday_lib}, データ={is_holiday_data}")
    
    # adjusted_powerの確認
    if "adjusted_power" in area_df.columns:
        power_missing = area_df["adjusted_power"].isnull().sum()
        power_total = len(area_df)
        print(f"\n⚡ adjusted_power確認:")
        print(f"  欠損値: {power_missing}件 ({power_missing/power_total*100:.1f}%)")
        
        if power_missing > 0:
            print(f"  ⚠️ adjusted_powerに欠損値があります")
            # 欠損値の原因調査
            zones_with_missing = area_df[area_df["adjusted_power"].isnull()]["zone"].unique()
            print(f"  欠損があるゾーン: {zones_with_missing}")
        else:
            print(f"  ✅ adjusted_powerに欠損値なし")
else:
    print("❌ 制御エリア集約に失敗")
    area_df = None



🔧 制御エリア集約...

[AreaAggregator] エリア 'Area 1' のカテゴリカル変数処理開始
[AreaAggregator] Area 1 - A/C ON/OFF 処理中...
[AreaAggregator] Area 1 - A/C ON/OFF ユニーク値: {'OFF': 699212, 'ON': 247864}
[AreaAggregator] Area 1 - A/C Mode 処理中...
[AreaAggregator] Area 1 - A/C Mode ユニーク値: {'COOL': 416919, 'HEAT': 363987, 'FAN': 126583, '-': 39587}
[AreaAggregator] Area 1 - A/C Fan Speed 処理中...
[AreaAggregator] Area 1 - A/C Fan Speed ユニーク値: {'-': 699212, 'Low': 205122, 'Medium': 21569, 'High': 21172, 'Auto': 1}

[AreaAggregator] エリア別マッピングログ保存: logs/preprocessing/unknown/zone_mapping_log_20251002_154025.json
[AreaAggregator] Zone Area 1: Processing 7 outdoor units
[AreaAggregator] Found 508731 records for Mesh ID: 49-1
  Total_kWh統計: 平均=25694.39, 最大=166985.00
  adjusted_power統計: 平均=25694.39, 最大=166985.00
[AreaAggregator] Found 508731 records for Mesh ID: 49-2
  Total_kWh統計: 平均=25694.39, 最大=166985.00
  adjusted_power統計: 平均=25694.39, 最大=166985.00
[AreaAggregator] Found 508731 records for Mesh ID: 49-3
  Total_kWh統計: 平

In [10]:
# 前処理済みデータの保存
if area_df is not None:
    # 前処理済みデータを保存
    preprocessor.save(ac_processed, pm_processed)
    
    # 集約済みデータも保存
    area_output_path = f"data/02_PreprocessedData/{store_name}/features_processed_{store_name}.csv"
    os.makedirs(os.path.dirname(area_output_path), exist_ok=True)
    area_df.to_csv(area_output_path, index=False, encoding="utf-8-sig")
    print(f"✅ 前処理済みデータ保存完了: {area_output_path}")
    
    # データの基本統計情報
    print(f"\n📊 データの基本統計情報:")
    print(area_df.describe())
    
    print(f"\n🔍 欠損値の確認:")
    missing_data = area_df.isnull().sum()
    missing_data = missing_data[missing_data > 0]
    if len(missing_data) > 0:
        print(missing_data)
    else:
        print("✅ 欠損値なし")
else:
    print("❌ データが読み込まれていません")


✅ 前処理済みデータ保存完了: data/02_PreprocessedData/Clea/features_processed_Clea.csv

📊 データの基本統計情報:
       A/C Set Temperature  Indoor Temp.    A/C ON/OFF      A/C Mode  \
count         45707.000000  45707.000000  45707.000000  45707.000000   
mean             24.924847     23.921647      0.215941      1.077034   
std               1.417064      5.778953      0.411478      1.397043   
min              18.000000      6.000000      0.000000      0.000000   
25%              25.000000     20.500000      0.000000      0.000000   
50%              25.000000     25.100000      0.000000      0.000000   
75%              26.000000     27.700000      0.000000      3.000000   
max              30.000000     44.900000      1.000000      3.000000   

       A/C Fan Speed  adjusted_power     DayOfWeek          Hour  \
count   45707.000000    6.008400e+04  64531.000000  64531.000000   
mean        1.038200    7.716964e+04      2.996343     11.483706   
std         0.207362    1.499877e+05      1.997221      6.

In [11]:
# データの基本統計情報
if area_df is not None:
    print("📊 データの基本統計情報:")
    print(area_df.describe())
    
    print("\n🔍 欠損値の確認:")
    missing_data = area_df.isnull().sum()
    missing_data = missing_data[missing_data > 0]
    if len(missing_data) > 0:
        print(missing_data)
    else:
        print("✅ 欠損値なし")


📊 データの基本統計情報:
       A/C Set Temperature  Indoor Temp.    A/C ON/OFF      A/C Mode  \
count         45707.000000  45707.000000  45707.000000  45707.000000   
mean             24.924847     23.921647      0.215941      1.077034   
std               1.417064      5.778953      0.411478      1.397043   
min              18.000000      6.000000      0.000000      0.000000   
25%              25.000000     20.500000      0.000000      0.000000   
50%              25.000000     25.100000      0.000000      0.000000   
75%              26.000000     27.700000      0.000000      3.000000   
max              30.000000     44.900000      1.000000      3.000000   

       A/C Fan Speed  adjusted_power     DayOfWeek          Hour  \
count   45707.000000    6.008400e+04  64531.000000  64531.000000   
mean        1.038200    7.716964e+04      2.996343     11.483706   
std         0.207362    1.499877e+05      1.997221      6.923108   
min         1.000000    7.000000e+01      0.000000      0.000000 

In [12]:
# ゾーン別データの確認
if area_df is not None:
    print("🏢 ゾーン別データ確認:")
    for zone in area_df['zone'].unique():
        zone_data = area_df[area_df['zone'] == zone]
        print(f"\n📍 {zone}:")
        print(f"  - レコード数: {len(zone_data):,}")
        print(f"  - 期間: {zone_data['datetime'].min()} ～ {zone_data['datetime'].max()}")
        
        # 電力データの確認
        power_data = zone_data['adjusted_power'].dropna()
        if len(power_data) > 0:
            print(f"  - 電力データ: {len(power_data):,}件 (平均: {power_data.mean():.1f}W)")
        else:
            print(f"  - 電力データ: なし")
        
        # 室温データの確認
        temp_data = zone_data['Indoor Temp.'].dropna()
        if len(temp_data) > 0:
            print(f"  - 室温データ: {len(temp_data):,}件 (平均: {temp_data.mean():.1f}°C)")
        else:
            print(f"  - 室温データ: なし")


🏢 ゾーン別データ確認:

📍 Area 1:
  - レコード数: 10,654


KeyError: 'datetime'

## 2. 特徴量エンジニアリング

モデル学習用の特徴量を準備し、特徴量の重要度を確認します。


In [ ]:
# 特徴量の準備
if area_df is not None:
    # 利用可能な特徴量の確認
    base_feats = [
        "A/C Set Temperature",
        "Indoor Temp. Lag1", 
        "A/C ON/OFF",
        "A/C Mode",
        "A/C Fan Speed",
        "Outdoor Temp.",
        "Outdoor Humidity",
        "Solar Radiation",
        "DayOfWeek",
        "Hour",
        "Month",
        "IsWeekend",
        "IsHoliday"
    ]
    
    available_feats = [col for col in base_feats if col in area_df.columns]
    missing_feats = [col for col in base_feats if col not in area_df.columns]
    
    print(f"✅ 利用可能な特徴量 ({len(available_feats)}個):")
    for feat in available_feats:
        print(f"  - {feat}")
    
    if missing_feats:
        print(f"\n⚠️ 不足している特徴量 ({len(missing_feats)}個):")
        for feat in missing_feats:
            print(f"  - {feat}")
    
    # 特徴量の相関確認
    print(f"\n🔍 特徴量間の相関確認:")
    corr_matrix = area_df[available_feats + ['Indoor Temp.', 'adjusted_power']].corr()
    
    # 室温との相関
    if 'Indoor Temp.' in corr_matrix.columns:
        temp_corr = corr_matrix['Indoor Temp.'].drop('Indoor Temp.').abs().sort_values(ascending=False)
        print(f"\n🌡️ 室温との相関 (上位10位):")
        for feat, corr in temp_corr.head(10).items():
            print(f"  {feat}: {corr:.3f}")
    
    # 電力との相関
    if 'adjusted_power' in corr_matrix.columns:
        power_corr = corr_matrix['adjusted_power'].drop('adjusted_power').abs().sort_values(ascending=False)
        print(f"\n⚡ 電力との相関 (上位10位):")
        for feat, corr in power_corr.head(10).items():
            print(f"  {feat}: {corr:.3f}")


✅ 利用可能な特徴量 (10個):
  - A/C Set Temperature
  - Indoor Temp. Lag1
  - A/C ON/OFF
  - A/C Mode
  - A/C Fan Speed
  - DayOfWeek
  - Hour
  - Month
  - IsWeekend
  - IsHoliday

⚠️ 不足している特徴量 (3個):
  - Outdoor Temp.
  - Outdoor Humidity
  - Solar Radiation

🔍 特徴量間の相関確認:

🌡️ 室温との相関 (上位10位):
  Indoor Temp. Lag1: 0.980
  A/C Mode: 0.620
  Month: 0.346
  Hour: 0.230
  A/C ON/OFF: 0.173
  adjusted_power: 0.097
  A/C Set Temperature: 0.091
  A/C Fan Speed: 0.062
  IsHoliday: 0.053
  IsWeekend: 0.016

⚡ 電力との相関 (上位10位):
  A/C ON/OFF: 0.544
  Hour: 0.161
  A/C Fan Speed: 0.161
  A/C Set Temperature: 0.150
  IsWeekend: 0.146
  DayOfWeek: 0.119
  Indoor Temp.: 0.097
  Indoor Temp. Lag1: 0.068
  IsHoliday: 0.067
  A/C Mode: 0.034


## 3. 予測モデル構築

電力・環境予測モデルを構築し、性能を評価します。


In [ ]:
# モデルビルダーの初期化
if area_df is not None:
    builder = ModelBuilder(store_name)
    
    print("🔧 モデル構築開始...")
    start_time = time.time()
    
    # モデル学習
    models = builder.train_by_zone(area_df, master)
    
    end_time = time.time()
    print(f"✅ モデル構築完了 (処理時間: {end_time - start_time:.2f}秒)")
    print(f"📊 構築されたモデル数: {len(models)}")
    
    # 各ゾーンのモデル性能確認
    for zone_name, model_pack in models.items():
        print(f"\n📍 {zone_name}:")
        print(f"  - 特徴量数: {len(model_pack.feature_cols)}")
        print(f"  - 温度モデル: {'✅' if model_pack.temp_model else '❌'}")
        print(f"  - 電力モデル: {'✅' if model_pack.power_model else '❌'}")
        print(f"  - マルチアウトプットモデル: {'✅' if model_pack.multi_output_model else '❌'}")
else:
    print("❌ データが読み込まれていません")
    models = None


🔧 モデル構築開始...
[ModelBuilder] Starting train_by_zone. Input shape: (64531, 14)
[ModelBuilder] Found zones: ['Area 1', 'Area 2', 'Area 3', 'Area 4', 'Break Room', 'Meeting Room']
[ModelBuilder] Zone Area 1: 10654 records
[ModelBuilder] Zone Area 1: Checking adjusted_power column...
[ModelBuilder] Zone Area 1: adjusted_power non-null values: 8512
[Model] Area 1 Temp: MAE=0.42 R2=0.989
[Model] Area 1 Power: MAE=44819.2 R2=0.869
[ModelBuilder] Zone Area 2: 10767 records
[ModelBuilder] Zone Area 2: Checking adjusted_power column...
[ModelBuilder] Zone Area 2: adjusted_power non-null values: 10765
[Model] Area 2 Temp: MAE=0.42 R2=0.983
[Model] Area 2 Power: MAE=49853.6 R2=0.786
[ModelBuilder] Zone Area 3: 10654 records
[ModelBuilder] Zone Area 3: Checking adjusted_power column...
[ModelBuilder] Zone Area 3: adjusted_power non-null values: 8512
[Model] Area 3 Temp: MAE=0.46 R2=0.984
[Model] Area 3 Power: MAE=6663.7 R2=0.850
[ModelBuilder] Zone Area 4: 10894 records
[ModelBuilder] Zone Area 4: C

In [ ]:
# モデル性能の詳細評価
if models is not None:
    print("📊 モデル性能詳細評価:")
    
    for zone_name, model_pack in models.items():
        zone_data = area_df[area_df['zone'] == zone_name]
        
        print(f"\n📍 {zone_name}:")
        
        # 温度モデルの評価
        if model_pack.temp_model and 'Indoor Temp.' in zone_data.columns:
            # テストデータでの評価
            X_temp = zone_data[model_pack.feature_cols].replace([np.inf, -np.inf], np.nan).fillna(0)
            y_temp = zone_data['Indoor Temp.'].astype(float)
            mask = y_temp.notna()
            
            if mask.sum() > 0:
                X_temp_clean = X_temp[mask]
                y_temp_clean = y_temp[mask]
                
                # 予測
                y_pred_temp = model_pack.temp_model.predict(X_temp_clean)
                
                # 評価指標
                mae_temp = mean_absolute_error(y_temp_clean, y_pred_temp)
                r2_temp = r2_score(y_temp_clean, y_pred_temp)
                
                print(f"  🌡️ 温度モデル:")
                print(f"    - MAE: {mae_temp:.2f}°C")
                print(f"    - R²: {r2_temp:.3f}")
                print(f"    - 予測範囲: {y_pred_temp.min():.1f}°C ～ {y_pred_temp.max():.1f}°C")
        
        # 電力モデルの評価
        if model_pack.power_model and 'adjusted_power' in zone_data.columns:
            X_power = zone_data[model_pack.feature_cols].replace([np.inf, -np.inf], np.nan).fillna(0)
            y_power = zone_data['adjusted_power'].astype(float)
            mask = y_power.notna()
            
            if mask.sum() > 0:
                X_power_clean = X_power[mask]
                y_power_clean = y_power[mask]
                
                # 予測
                y_pred_power = model_pack.power_model.predict(X_power_clean)
                
                # 評価指標
                mae_power = mean_absolute_error(y_power_clean, y_pred_power)
                r2_power = r2_score(y_power_clean, y_pred_power)
                
                print(f"  ⚡ 電力モデル:")
                print(f"    - MAE: {mae_power:.1f}W")
                print(f"    - R²: {r2_power:.3f}")
                print(f"    - 予測範囲: {y_pred_power.min():.1f}W ～ {y_pred_power.max():.1f}W")


📊 モデル性能詳細評価:

📍 Area 1:
  🌡️ 温度モデル:
    - MAE: 0.21°C
    - R²: 0.996
    - 予測範囲: 8.3°C ～ 40.7°C
  ⚡ 電力モデル:
    - MAE: 25797.9W
    - R²: 0.939
    - 予測範囲: 15429.2W ～ 1094821.7W

📍 Area 2:
  🌡️ 温度モデル:
    - MAE: 0.20°C
    - R²: 0.995
    - 予測範囲: 12.4°C ～ 42.4°C
  ⚡ 電力モデル:
    - MAE: 27211.8W
    - R²: 0.909
    - 予測範囲: 2933.8W ～ 1073459.1W

📍 Area 3:
  🌡️ 温度モデル:
    - MAE: 0.23°C
    - R²: 0.995
    - 予測範囲: 8.6°C ～ 36.3°C
  ⚡ 電力モデル:
    - MAE: 3837.1W
    - R²: 0.934
    - 予測範囲: 2059.2W ～ 155441.0W

📍 Area 4:
  🌡️ 温度モデル:
    - MAE: 0.19°C
    - R²: 0.995
    - 予測範囲: 12.9°C ～ 36.3°C
  ⚡ 電力モデル:
    - MAE: 9340.6W
    - R²: 0.887
    - 予測範囲: 2191.1W ～ 421506.0W

📍 Break Room:
  🌡️ 温度モデル:
    - MAE: 0.37°C
    - R²: 0.985
    - 予測範囲: 7.4°C ～ 37.8°C
  ⚡ 電力モデル:
    - MAE: 9230.4W
    - R²: 0.657
    - 予測範囲: 1250.3W ～ 145677.7W

📍 Meeting Room:
  🌡️ 温度モデル:
    - MAE: 0.25°C
    - R²: 0.998
    - 予測範囲: 6.2°C ～ 44.4°C
  ⚡ 電力モデル:
    - MAE: 9188.9W
    - R²: 0.656
    - 予測範囲: 1234.2W ～ 143715.5

## 4. 制御値探索

最適制御値を探索し、探索過程を可視化します。


In [ ]:
# 最適化期間の設定
if models is not None:
    # 最適化期間（今日から3日後まで）
    today = pd.Timestamp.today().normalize()
    start_date = today.strftime("%Y-%m-%d")
    end_date = (today + pd.Timedelta(days=3)).strftime("%Y-%m-%d")
    
    print(f"🎯 最適化期間: {start_date} ～ {end_date}")
    
    # 日時範囲の生成
    date_range = pd.date_range(
        start=pd.to_datetime(start_date), 
        end=pd.to_datetime(end_date), 
        freq="1H"
    )
    date_range = date_range[(date_range.hour >= 0) & (date_range.hour <= 23)]
    
    print(f"⏰ 最適化対象時間数: {len(date_range)}時間")
    print(f"📅 期間: {date_range[0]} ～ {date_range[-1]}")
else:
    print("❌ モデルが構築されていません")


🎯 最適化期間: 2025-10-02 ～ 2025-10-05
⏰ 最適化対象時間数: 73時間
📅 期間: 2025-10-02 00:00:00 ～ 2025-10-05 00:00:00


In [ ]:
# 天気予報データの準備（ダミーデータ）
if models is not None:
    # 簡単な天気予報データを生成
    weather_data = []
    for timestamp in date_range:
        # 季節に応じた温度変化
        base_temp = 20 + 10 * np.sin(2 * np.pi * timestamp.dayofyear / 365)
        daily_variation = 5 * np.sin(2 * np.pi * timestamp.hour / 24)
        temp = base_temp + daily_variation + np.random.normal(0, 1)
        
        # 湿度（40-80%の範囲）
        humidity = 60 + 20 * np.sin(2 * np.pi * timestamp.hour / 24) + np.random.normal(0, 5)
        humidity = np.clip(humidity, 40, 80)
        
        # 日射量（昼間のみ）
        if 6 <= timestamp.hour <= 18:
            solar = 500 * np.sin(np.pi * (timestamp.hour - 6) / 12) + np.random.normal(0, 50)
            solar = max(0, solar)
        else:
            solar = 0
        
        weather_data.append({
            'datetime': timestamp,
            'Outdoor Temp.': temp,
            'Outdoor Humidity': humidity,
            'Solar Radiation': solar
        })
    
    weather_df = pd.DataFrame(weather_data)
    
    print(f"🌤️ 天気予報データ生成完了")
    print(f"📊 形状: {weather_df.shape}")
    print(f"🌡️ 外気温範囲: {weather_df['Outdoor Temp.'].min():.1f}°C ～ {weather_df['Outdoor Temp.'].max():.1f}°C")
    print(f"💧 湿度範囲: {weather_df['Outdoor Humidity'].min():.1f}% ～ {weather_df['Outdoor Humidity'].max():.1f}%")
    print(f"☀️ 日射量範囲: {weather_df['Solar Radiation'].min():.1f}W/m² ～ {weather_df['Solar Radiation'].max():.1f}W/m²")


🌤️ 天気予報データ生成完了
📊 形状: (73, 4)
🌡️ 外気温範囲: 3.9°C ～ 16.6°C
💧 湿度範囲: 40.0% ～ 80.0%
☀️ 日射量範囲: 0.0W/m² ～ 590.1W/m²


In [ ]:
# 期間最適化の実行
if models is not None and weather_df is not None:
    print("🔍 期間最適化開始...")
    
    # 期間最適化器の初期化
    optimizer = PeriodOptimizer(master, models, max_workers=6)
    
    start_time = time.time()
    
    # 最適化実行
    schedule = optimizer.optimize_period(date_range, weather_df, preference="energy")
    
    end_time = time.time()
    
    print(f"✅ 期間最適化完了 (処理時間: {end_time - start_time:.2f}秒)")
    print(f"📊 最適化されたゾーン数: {len(schedule)}")
    
    # 各ゾーンの最適化結果確認
    for zone_name, zone_schedule in schedule.items():
        print(f"\n📍 {zone_name}:")
        print(f"  - スケジュール数: {len(zone_schedule)}")
        
        # 制御値の統計
        set_temps = [s['set_temp'] for s in zone_schedule.values()]
        modes = [s['mode'] for s in zone_schedule.values()]
        fans = [s['fan'] for s in zone_schedule.values()]
        pred_temps = [s['pred_temp'] for s in zone_schedule.values()]
        pred_powers = [s['pred_power'] for s in zone_schedule.values()]
        
        print(f"  - 設定温度: {min(set_temps)}°C ～ {max(set_temps)}°C (平均: {np.mean(set_temps):.1f}°C)")
        print(f"  - 運転モード: {min(modes)} ～ {max(modes)} (最頻値: {max(set(modes), key=modes.count)})")
        print(f"  - ファン速度: {min(fans)} ～ {max(fans)} (最頻値: {max(set(fans), key=fans.count)})")
        print(f"  - 予測室温: {min(pred_temps):.1f}°C ～ {max(pred_temps):.1f}°C (平均: {np.mean(pred_temps):.1f}°C)")
        print(f"  - 予測電力: {min(pred_powers):.1f}W ～ {max(pred_powers):.1f}W (合計: {np.sum(pred_powers):.1f}W)")
else:
    print("❌ 最適化に必要なデータが不足しています")
    schedule = None


🔍 期間最適化開始...
[PeriodOptimizer] Starting period optimization for 73 hours
[PeriodOptimizer] Date range: 2025-10-02 00:00:00 to 2025-10-05 00:00:00
[PeriodOptimizer] Preference: energy
[PeriodOptimizer] Max workers: 6
[PeriodOptimizer] Weights - Comfort: 0.2, Power: 0.8
[PeriodOptimizer] Available zones: ['Area 1', 'Area 2', 'Area 3', 'Area 4', 'Break Room', 'Meeting Room']
[PeriodOptimizer] Starting period optimization for zone: Area 1
[PeriodOptimizer] Zone Area 1: Beam search with width=5, candidates=8×3×3
[PeriodOptimizer] Starting period optimization for zone: Area 2
[PeriodOptimizer] Zone Area 2: Beam search with width=5, candidates=7×3×3
[PeriodOptimizer] Starting period optimization for zone: Area 3
[PeriodOptimizer] Zone Area 3: Beam search with width=5, candidates=8×3×3
[PeriodOptimizer] Starting period optimization for zone: Area 4
[PeriodOptimizer] Zone Area 4: Beam search with width=5, candidates=4×3×3
[PeriodOptimizer] Starting period optimization for zone: Break Room
[Peri

## 5. 結果出力・可視化

最適化結果を可視化し、分析します。


In [ ]:
# 最適化結果の可視化
if schedule is not None:
    # 各ゾーンの結果を可視化
    for zone_name, zone_schedule in schedule.items():
        # データの準備
        timestamps = list(zone_schedule.keys())
        set_temps = [zone_schedule[t]['set_temp'] for t in timestamps]
        pred_temps = [zone_schedule[t]['pred_temp'] for t in timestamps]
        pred_powers = [zone_schedule[t]['pred_power'] for t in timestamps]
        modes = [zone_schedule[t]['mode'] for t in timestamps]
        fans = [zone_schedule[t]['fan'] for t in timestamps]
        
        # サブプロットの作成
        fig = make_subplots(
            rows=4, cols=1,
            subplot_titles=[
                f"{zone_name} - 設定温度・予測室温",
                f"{zone_name} - 予測電力",
                f"{zone_name} - 運転モード",
                f"{zone_name} - ファン速度"
            ],
            vertical_spacing=0.08
        )
        
        # 設定温度・予測室温
        fig.add_trace(
            go.Scatter(x=timestamps, y=set_temps, name='設定温度', line=dict(color='red', width=2)),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=timestamps, y=pred_temps, name='予測室温', line=dict(color='blue', width=2)),
            row=1, col=1
        )
        
        # 予測電力
        fig.add_trace(
            go.Scatter(x=timestamps, y=pred_powers, name='予測電力', line=dict(color='green', width=2)),
            row=2, col=1
        )
        
        # 運転モード
        fig.add_trace(
            go.Scatter(x=timestamps, y=modes, name='運転モード', mode='markers', marker=dict(color='orange', size=6)),
            row=3, col=1
        )
        
        # ファン速度
        fig.add_trace(
            go.Scatter(x=timestamps, y=fans, name='ファン速度', mode='markers', marker=dict(color='purple', size=6)),
            row=4, col=1
        )
        
        # レイアウトの設定
        fig.update_layout(
            title=f"{zone_name} 最適化結果",
            height=800,
            showlegend=True
        )
        
        # 軸ラベルの設定
        fig.update_xaxes(title_text="時刻", row=4, col=1)
        fig.update_yaxes(title_text="温度 (°C)", row=1, col=1)
        fig.update_yaxes(title_text="電力 (W)", row=2, col=1)
        fig.update_yaxes(title_text="モード", row=3, col=1)
        fig.update_yaxes(title_text="ファン速度", row=4, col=1)
        
        fig.show()
        
        print(f"✅ {zone_name} の可視化完了")
else:
    print("❌ 最適化結果がありません")


✅ Area 4 の可視化完了


✅ Break Room の可視化完了


✅ Area 2 の可視化完了


✅ Meeting Room の可視化完了


✅ Area 3 の可視化完了


✅ Area 1 の可視化完了


In [ ]:
# 天気予報データの可視化
if weather_df is not None:
    fig = make_subplots(
        rows=3, cols=1,
        subplot_titles=[
            "外気温",
            "外気湿度",
            "日射量"
        ],
        vertical_spacing=0.08
    )
    
    # 外気温
    fig.add_trace(
        go.Scatter(x=weather_df['datetime'], y=weather_df['Outdoor Temp.'], name='外気温', line=dict(color='orange', width=2)),
        row=1, col=1
    )
    
    # 外気湿度
    fig.add_trace(
        go.Scatter(x=weather_df['datetime'], y=weather_df['Outdoor Humidity'], name='外気湿度', line=dict(color='lightblue', width=2)),
        row=2, col=1
    )
    
    # 日射量
    fig.add_trace(
        go.Scatter(x=weather_df['datetime'], y=weather_df['Solar Radiation'], name='日射量', line=dict(color='yellow', width=2)),
        row=3, col=1
    )
    
    # レイアウトの設定
    fig.update_layout(
        title="天気予報データ",
        height=600,
        showlegend=True
    )
    
    # 軸ラベルの設定
    fig.update_xaxes(title_text="時刻", row=3, col=1)
    fig.update_yaxes(title_text="温度 (°C)", row=1, col=1)
    fig.update_yaxes(title_text="湿度 (%)", row=2, col=1)
    fig.update_yaxes(title_text="日射量 (W/m²)", row=3, col=1)
    
    fig.show()
    print("✅ 天気予報データの可視化完了")


✅ 天気予報データの可視化完了


In [ ]:
# 最適化結果のCSV出力
if schedule is not None:
    print("📁 最適化結果のCSV出力...")
    
    # 出力ディレクトリの作成
    output_dir = f"data/04_OutputData/{store_name}"
    os.makedirs(output_dir, exist_ok=True)
    
    # プランナーの初期化
    planner = Planner()
    
    # スケジュールの出力
    planner.export(schedule, output_dir)
    
    print(f"✅ 最適化結果を出力しました: {output_dir}")
    
    # 天気予報データの出力
    weather_output_path = os.path.join(output_dir, "weather_forecast.csv")
    weather_df.to_csv(weather_output_path, index=False, encoding="utf-8-sig")
    print(f"✅ 天気予報データを出力しました: {weather_output_path}")
    
    # 出力ファイルの確認
    print(f"\n📋 出力ファイル:")
    for file in os.listdir(output_dir):
        if file.endswith('.csv'):
            file_path = os.path.join(output_dir, file)
            file_size = os.path.getsize(file_path)
            print(f"  - {file} ({file_size:,} bytes)")
else:
    print("❌ 最適化結果がありません")


📁 最適化結果のCSV出力...


TypeError: Planner.__init__() missing 2 required positional arguments: 'store_name' and 'master'

## 6. 改善提案・次のステップ

各ステップの結果を基に、改善提案を行います。


In [ ]:
# 改善提案の生成
print("🔍 改善提案・次のステップ:")
print("\n1. データ品質の改善:")
if area_df is not None:
    missing_data = area_df.isnull().sum()
    missing_data = missing_data[missing_data > 0]
    if len(missing_data) > 0:
        print(f"   - 欠損値の補完: {missing_data.index.tolist()}")
    else:
        print("   - データ品質は良好です")

print("\n2. 特徴量エンジニアリングの改善:")
print("   - ラグ特徴量の追加（過去の温度・電力の影響）")
print("   - 移動平均特徴量の追加")
print("   - 外気温との差分特徴量の追加")

print("\n3. モデル性能の改善:")
if models is not None:
    for zone_name, model_pack in models.items():
        if model_pack.temp_model:
            print(f"   - {zone_name}: 温度モデルの性能向上")
        if model_pack.power_model:
            print(f"   - {zone_name}: 電力モデルの性能向上")

print("\n4. 最適化アルゴリズムの改善:")
print("   - ビーム幅の調整")
print("   - 制約条件の追加")
print("   - 多目的最適化の実装")

print("\n5. 可視化・分析の改善:")
print("   - 実績データとの比較")
print("   - 感度分析の追加")
print("   - コスト分析の追加")

print("\n✅ 改善提案完了")


## 7. 実行ログ・デバッグ情報

実行過程でのログやデバッグ情報を記録します。


In [ ]:
# 実行ログの記録
print("📝 実行ログ・デバッグ情報:")
print(f"\n🕐 実行日時: {pd.Timestamp.now()}")
print(f"🏢 対象ストア: {store_name}")
print(f"📊 データ形状: {area_df.shape if area_df is not None else 'N/A'}")
print(f"🔧 モデル数: {len(models) if models is not None else 0}")
print(f"⏰ 最適化時間数: {len(date_range) if 'date_range' in locals() else 0}")
print(f"📈 スケジュール数: {len(schedule) if schedule is not None else 0}")

# メモリ使用量の確認
try:
    import psutil
    memory_info = psutil.virtual_memory()
    print(f"\n💾 メモリ使用量: {memory_info.percent:.1f}% ({memory_info.used / 1024**3:.1f}GB / {memory_info.total / 1024**3:.1f}GB)")
except ImportError:
    print("\n💾 メモリ使用量: psutilがインストールされていません")

print("\n✅ 実行ログ記録完了")
